# 1. Import Library

In [1]:
import pandas as pd

# 2. Load Data

In [2]:
df = pd.read_csv("ai_job_dataset.csv")
df.head()

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


Notes:
* experience_level:
    - SE = Senior
    - EN = Entry
    - MI = Mid
    - EX = Executive
* employment_type:
    - CT = Contract
    - FL = Full-Time
    - PT = Part-Time
    - FT = Freelance
* company_size:
    - M = Medium (50-250)
    - L = Large (>250)
    - S = Small (<50)
* remote_ratio:
    - 0 = No remote
    - 50 = Hybrid
    - 100 = Remote

In [3]:
summary = pd.DataFrame({
    'Features': df.columns,
    'Number of Cardinality': [df[col].nunique() for col in df.columns],
    'Cardinality': [df[col].unique()[:20]for col in df.columns]
})

print(summary)

                  Features  Number of Cardinality  \
0                   job_id                  15000   
1                job_title                     20   
2               salary_usd                  14315   
3          salary_currency                      3   
4         experience_level                      4   
5          employment_type                      4   
6         company_location                     20   
7             company_size                      3   
8       employee_residence                     20   
9             remote_ratio                      3   
10         required_skills                  13663   
11      education_required                      4   
12        years_experience                     20   
13                industry                     15   
14            posting_date                    486   
15    application_deadline                    543   
16  job_description_length                   2000   
17          benefits_score                    

## Checking Data Type of Each Features

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 19 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   job_id                  15000 non-null  object 
 1   job_title               15000 non-null  object 
 2   salary_usd              15000 non-null  int64  
 3   salary_currency         15000 non-null  object 
 4   experience_level        15000 non-null  object 
 5   employment_type         15000 non-null  object 
 6   company_location        15000 non-null  object 
 7   company_size            15000 non-null  object 
 8   employee_residence      15000 non-null  object 
 9   remote_ratio            15000 non-null  int64  
 10  required_skills         15000 non-null  object 
 11  education_required      15000 non-null  object 
 12  years_experience        15000 non-null  int64  
 13  industry                15000 non-null  object 
 14  posting_date            15000 non-null

The data type of each features is correct

## Checking Duplicates Data

In [5]:
df.duplicated().sum()

0

There is no duplicate data

# 3. Statistical Analysis

In [6]:
df.describe()

,salary_usd,remote_ratio,years_experience,job_description_length,benefits_score
count,15000.000000,15000.000000,15000.000000,15000.000000,15000.000000
mean,115348.965133,49.483333,6.253200,1503.314733,7.504273
std,60260.940438,40.812712,5.545768,576.127083,1.450870
min,32519.000000,0.000000,0.000000,500.000000,5.000000
25%,70179.750000,0.000000,2.000000,1003.750000,6.200000
50%,99705.000000,50.000000,5.000000,1512.000000,7.500000
75%,146408.500000,100.000000,10.000000,2000.000000,8.800000
max,399095.000000,100.000000,19.000000,2499.000000,10.000000


## Normality Check

In [7]:
numeric_cols = df.select_dtypes(include='number').columns

for col in numeric_cols:
    print(f'{col}: {df[col].skew()}')

salary_usd: 1.2527592133197596
remote_ratio: 0.018981197240634334
years_experience: 0.7788132121793866
job_description_length: -0.01182202766151641
benefits_score: -0.009657871296079869


Features salary_usd is Positive Highly Skewed. Features remote_ratio, job_description_length, and benefits_score are aprroximately symetric, and Features years_experience is moderately skewed

In [8]:
for col in numeric_cols:
    print(f'{col}: {df[col].kurtosis()}')

salary_usd: 1.56922226950801
remote_ratio: -1.498868689442591
years_experience: -0.5249559915299491
job_description_length: -1.1979808763291446
benefits_score: -1.2089971234531076


Features salary_usd is leprokurtic. Features remote_ratio, years_experience, job_description_length, amd benefit_score are platykurtic

# 4. Eploratory Data Analysist

In [9]:
df.head(5)

,job_id,job_title,salary_usd,salary_currency,experience_level,employment_type,company_location,company_size,employee_residence,remote_ratio,required_skills,education_required,years_experience,industry,posting_date,application_deadline,job_description_length,benefits_score,company_name
0,AI00001,AI Research Scientist,90376,USD,SE,CT,China,M,China,50,"Tableau, PyTorch, Kubernetes, Linux, NLP",Bachelor,9,Automotive,2024-10-18,2024-11-07,1076,5.9,Smart Analytics
1,AI00002,AI Software Engineer,61895,USD,EN,CT,Canada,M,Ireland,100,"Deep Learning, AWS, Mathematics, Python, Docker",Master,1,Media,2024-11-20,2025-01-11,1268,5.2,TechCorp Inc
2,AI00003,AI Specialist,152626,USD,MI,FL,Switzerland,L,South Korea,0,"Kubernetes, Deep Learning, Java, Hadoop, NLP",Associate,2,Education,2025-03-18,2025-04-07,1974,9.4,Autonomous Tech
3,AI00004,NLP Engineer,80215,USD,SE,FL,India,M,India,50,"Scala, SQL, Linux, Python",PhD,7,Consulting,2024-12-23,2025-02-24,1345,8.6,Future Systems
4,AI00005,AI Consultant,54624,EUR,EN,PT,France,S,Singapore,100,"MLOps, Java, Tableau, Python",Master,0,Media,2025-04-15,2025-06-23,1989,6.6,Advanced Robotics


## A. What Is The Highest Paying Job?

In [11]:
highest_job = df.groupby('job_title')['salary_usd'].max()
highest_job

job_title
AI Architect                   398084
AI Consultant                  325757
AI Product Manager             381575
AI Research Scientist          394917
AI Software Engineer           379418
AI Specialist                  390292
Autonomous Systems Engineer    364585
Computer Vision Engineer       366957
Data Analyst                   361541
Data Engineer                  359044
Data Scientist                 388754
Deep Learning Engineer         362503
Head of AI                     361718
ML Ops Engineer                352006
Machine Learning Engineer      344427
Machine Learning Researcher    399095
NLP Engineer                   357880
Principal Data Scientist       364635
Research Scientist             339526
Robotics Engineer              346789
Name: salary_usd, dtype: int64

In [12]:
highest_average = df.groupby('job_title')['salary_usd'].mean().sort_values(ascending=False)
highest_average

job_title
AI Specialist                  120570.758242
Machine Learning Engineer      118827.919689
Head of AI                     118542.968627
AI Research Scientist          117897.925926
AI Architect                   117436.513619
Robotics Engineer              116470.793149
Principal Data Scientist       116305.219346
Research Scientist             116066.502695
Data Engineer                  115970.720961
Data Scientist                 115819.008333
ML Ops Engineer                115431.335172
AI Product Manager             114680.909825
AI Software Engineer           114273.201531
Deep Learning Engineer         113725.632312
AI Consultant                  113671.870739
Computer Vision Engineer       113472.707182
NLP Engineer                   112670.937008
Machine Learning Researcher    112621.747525
Autonomous Systems Engineer    111396.557272
Data Analyst                   111321.180501
Name: salary_usd, dtype: float64

As we can see, Robotic Engineer has the lowest salary in maximum. But if we're talking about average, Data Analyst placed as the lowest paying job in average. This phenomenon may influence by other factors too, for example Education Level  

In [15]:
job_ed = df.groupby('education_required')['salary_usd'].mean().sort_values(ascending=False)
job_ed

education_required
Master       117171.815902
Bachelor     115861.629190
Associate    114605.708058
PhD          113728.165579
Name: salary_usd, dtype: float64

In [16]:
df['education_required'].unique()

array(['Bachelor', 'Master', 'Associate', 'PhD'], dtype=object)

In [28]:
bachelor = df[df['education_required'] == 'Bachelor']
educ = bachelor.groupby('job_title')['salary_usd'].mean().sort_values(ascending=False)
print(f'Highest Average Paying Job for Bachelor Degree: ', educ)


Highest Average Paying Job for Bachelor Degree:  job_title
Machine Learning Engineer      125437.483333
Data Scientist                 125375.261780
AI Specialist                  123821.613095
Head of AI                     122473.897059
Computer Vision Engineer       119234.794444
Robotics Engineer              117914.515625
Data Engineer                  117622.169154
AI Software Engineer           116223.937799
AI Consultant                  115968.837349
AI Architect                   115310.400000
ML Ops Engineer                114524.030769
NLP Engineer                   114307.335079
Deep Learning Engineer         114262.010256
Principal Data Scientist       114005.779661
Machine Learning Researcher    113091.000000
Autonomous Systems Engineer    112487.259669
AI Product Manager             112194.284946
Research Scientist             108326.567708
Data Analyst                   108122.355769
AI Research Scientist          108036.860215
Name: salary_usd, dtype: float64


In [29]:
master = df[df['education_required'] == 'Master']
ma = master.groupby('job_title')['salary_usd'].mean().sort_values(ascending=False)
print(f'Highest Average Paying Job for Master Degree: ', ma)

Highest Average Paying Job for Master Degree:  job_title
AI Research Scientist          124687.869347
Head of AI                     123964.176471
AI Architect                   121636.565000
Machine Learning Engineer      121463.698324
Robotics Engineer              120974.073892
Data Analyst                   118279.217822
AI Product Manager             117755.373684
Research Scientist             117591.208092
Principal Data Scientist       117312.454545
Data Engineer                  117093.831461
ML Ops Engineer                117023.697297
AI Software Engineer           116759.529730
AI Specialist                  116429.424870
Machine Learning Researcher    115586.707071
Data Scientist                 115564.885870
NLP Engineer                   114165.584906
Deep Learning Engineer         114061.785714
Computer Vision Engineer       113779.270718
AI Consultant                  111253.523529
Autonomous Systems Engineer    106466.578378
Name: salary_usd, dtype: float64


Master graduate has earn higher income compared to those who graduated from PhD despite their education level

## B. Are Remote Workers Has Higher Income?

## C. How Freelance Workers Income Compared to Full Time Employee?

## D. Does Company Size Has Influence on Employee Income?

## E. From Which Country Has Highest Income Employee? Which Country Pay Highest For Their Employee?

In [10]:
df['employee_residence'].unique()

array(['China', 'Ireland', 'South Korea', 'India', 'Singapore', 'Germany',
       'United Kingdom', 'France', 'Austria', 'Sweden', 'Norway',
       'Israel', 'United States', 'Netherlands', 'Denmark', 'Switzerland',
       'Finland', 'Japan', 'Canada', 'Australia'], dtype=object)

## F. What Industry Has Highest Potential?